In [ ]:
import os
import numpy as np
import soundfile as sf
import librosa
from PIL import Image
import re

def scale_to_uint8(array):
    array = array.astype(float)
    if np.max(array) == np.min(array):
        return np.zeros(array.shape, dtype=np.uint8)
    scaled_array = (array - np.min(array)) / (np.max(array) - np.min(array))
    return (scaled_array * 255).astype(np.uint8)

In [ ]:
def spect_create(file_path, title='', duration_sec=30, image_dim=224, n_fft=1024, win_length=512, top_db=80):

    #win_length = 512
    #n_fft = 1024
    n_mels = image_dim # The final height of the image

    print(f"Pipeline: Direct generation at {image_dim}x{image_dim}")
    print(f"Analysis parameters: win_length={win_length}, n_fft={n_fft}, n_mels={n_mels}")

    match = re.search(r'-(\d{4})(\d{2})(\d{2})T', file_path)

    file_name = ''.join(match.groups())

    if title:
        output_folder = '-'.join(match.groups()) + f' spects {image_dim}px {title}'
    else:
        output_folder = '-'.join(match.groups()) + f' spects {image_dim}px'

    os.makedirs(output_folder, exist_ok=True)
    noise_folder_path = os.path.join(output_folder, "noise")
    os.makedirs(noise_folder_path, exist_ok=True)

    try:
        with sf.SoundFile(file_path, 'r') as audio_file:
            sr = audio_file.samplerate
            chunk_size_frames = int(duration_sec * sr)
            step_size_frames = chunk_size_frames // 2

            hop_length = int(round(chunk_size_frames / image_dim))
            total_frames = audio_file.frames

            print(f"Audio detected at {sr} Hz.")
            print(f"Calculated hop_length={hop_length} for an image width of {image_dim}")

            chunk_num = 0
            for start_frame in range(0, total_frames, step_size_frames):
                if start_frame + chunk_size_frames > total_frames:
                    break

                print(f"Processing chunk {chunk_num + 1}...")

                audio_file.seek(start_frame)
                y_chunk = audio_file.read(chunk_size_frames)
                peak_amplitude = np.max(np.abs(y_chunk))
                y_normalized = y_chunk / peak_amplitude if peak_amplitude > 0 else y_chunk

                S = librosa.feature.melspectrogram(
                    y=y_normalized, sr=sr, n_fft=n_fft, win_length=win_length,
                    hop_length=hop_length, n_mels=n_mels
                )[:, :]


                S_dB = librosa.power_to_db(S, ref=np.max, top_db=top_db)

                base_img_array = scale_to_uint8(S_dB)
                final_img_array = 255 - base_img_array # Invert colors
                final_img = Image.fromarray(final_img_array, 'L')

                # Create denoised image for entropy calculation
                S_dB_denoised = S_dB.copy()
                noise_floor = np.percentile(S_dB_denoised, 60)
                S_dB_denoised[S_dB_denoised < noise_floor] = noise_floor

                entropy_source_img = Image.fromarray(scale_to_uint8(S_dB_denoised))

                # Entropy calculation per column
                final_spec_data = np.array(entropy_source_img, dtype=np.float32)[15:, 3:-3]
                spec_for_entropy = final_spec_data - np.min(final_spec_data)
                col_sums = spec_for_entropy.sum(axis=0, keepdims=True) + 1e-12
                P = spec_for_entropy / col_sums
                H_t = -np.sum(P * np.log(P + 1e-12), axis=0)
                F = spec_for_entropy.shape[0]
                H_t_norm = H_t / np.log(F + 1e-12) if F > 1 else np.zeros_like(H_t)
                H_t_norm = np.clip(H_t_norm, 0.0, 1.0)

                # Entropy per bin
                bin_size = 8
                num_columns = H_t_norm.shape[0]
                if num_columns >= bin_size:
                    num_binned_columns = num_columns - (num_columns % bin_size)
                    truncated_H_t = H_t_norm[:num_binned_columns]
                    binned_entropies = truncated_H_t.reshape(-1, bin_size).mean(axis=1)
                else:
                    binned_entropies = H_t_norm

                alpha = 1
                entropy = np.exp(np.mean(np.log((H_t_norm + 1e-12) ** alpha))) ** (1 / alpha)

                is_signal = any(0 < h < 0.7 for h in H_t_norm)
                base_path = output_folder if is_signal else noise_folder_path

                print(f"--- Chunk {chunk_num+1:04d} ---")
                #print(f"  - Geometric Mean Entropy: {entropy:.6f} -> {'Signal' if is_signal else 'Noise'}")
                #print(f"  - Binned Entropies: {[f'{r:.4f}' for r in binned_entropies]}")

                output_filename = os.path.join(base_path, f"{file_name}_{chunk_num:04d}.png")
                final_img.save(output_filename)

                entropy_img_filename = os.path.join(base_path, f"{file_name}_{chunk_num:04d}_entropy_mask.png")

                # This is the mask used to calculate entropies, save for debugging only
                #entropy_source_img.save(entropy_img_filename)

                chunk_num += 1
                #if chunk_num > 1000:
                    #break

        print(f"\nProcessing complete. {chunk_num} spectrograms created.")

    except Exception as e:
        print(f"An error occurred: {e}")


# 30s spects

In [ ]:
if __name__ == '__main__':
    AUDIO_FILE = 'MARS-20230923T000000Z-2kHz.flac'

    spect_create(
        AUDIO_FILE,
        duration_sec=30,
        image_dim=224,
        n_fft=1024,
        win_length=512,
        top_db=80
    )

# 15s spects

In [ ]:
if __name__ == '__main__':
    spect_create(
        AUDIO_FILE,
        '15s',
        duration_sec=15,
        image_dim=224,
        n_fft=1024,
        win_length=512,
        top_db=80
    )